In [1]:
from pymongo import MongoClient

In [2]:
db = MongoClient('localhost', 27017).python_import

In [3]:
collection = db.fomc_articles

In [4]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [5]:
raw_corpus = ["Human machine interface for lab abc computer applications",
             "A survey of user opinion of computer system response time",
             "The EPS user interface management system",
             "System and human system engineering testing of EPS",              
             "Relation of user perceived response time to error measurement",
             "The generation of random binary unordered trees",
             "The intersection graph of paths in trees",
             "Graph minors IV Widths of trees and well quasi ordering",
             "Graph minors A survey"]

In [6]:
# Create a set of frequent words
stoplist = set('for a of the and to in'.split(' '))
# Lowercase each document, split it by white space and filter out stopwords
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in raw_corpus]

# Count word frequencies
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

# Only keep words that appear more than once
processed_corpus = [[token for token in text if frequency[token] > 1] for text in texts]
processed_corpus

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]

In [7]:
from gensim import corpora

dictionary = corpora.Dictionary(processed_corpus)
print(dictionary)

c:\program files\python35\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2017-03-02 10:25:29,859 : INFO : 'pattern' package not found; tag filters are not available for English
2017-03-02 10:25:29,885 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2017-03-02 10:25:29,891 : INFO : built Dictionary(12 unique tokens: ['user', 'time', 'survey', 'computer', 'graph']...) from 9 documents (total 29 corpus positions)


Dictionary(12 unique tokens: ['user', 'time', 'survey', 'computer', 'graph']...)


In [8]:
print(dictionary.token2id)

{'user': 3, 'time': 5, 'survey': 6, 'computer': 0, 'graph': 10, 'minors': 11, 'trees': 9, 'response': 4, 'eps': 8, 'system': 7, 'interface': 1, 'human': 2}


In [9]:
new_doc = "Human computer interaction"
new_vec = dictionary.doc2bow(new_doc.lower().split())
new_vec

[(0, 1), (2, 1)]

In [10]:
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]
bow_corpus

[[(0, 1), (1, 1), (2, 1)],
 [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(1, 1), (3, 1), (7, 1), (8, 1)],
 [(2, 1), (7, 2), (8, 1)],
 [(3, 1), (4, 1), (5, 1)],
 [(9, 1)],
 [(9, 1), (10, 1)],
 [(9, 1), (10, 1), (11, 1)],
 [(6, 1), (10, 1), (11, 1)]]

In [11]:
from gensim import models
# train the model
tfidf = models.TfidfModel(bow_corpus)
# transform the "system minors" string
tfidf[dictionary.doc2bow("system minors".lower().split())]

2017-03-02 10:25:30,669 : INFO : collecting document frequencies
2017-03-02 10:25:30,673 : INFO : PROGRESS: processing document #0
2017-03-02 10:25:30,676 : INFO : calculating IDF weights for 9 documents and 11 features (28 matrix non-zeros)


[(7, 0.5898341626740045), (11, 0.8075244024440723)]

In [12]:
dictionary.save('./corpus/test.dict')

2017-03-02 10:25:31,086 : INFO : saving Dictionary object under test.dict, separately None
2017-03-02 10:25:31,091 : INFO : saved test.dict


In [14]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('./corpus/corpus.mm', corpus)

2017-03-02 10:31:23,387 : INFO : storing corpus in Matrix Market format to corpus.mm
2017-03-02 10:31:23,390 : INFO : saving sparse matrix to corpus.mm
2017-03-02 10:31:23,392 : INFO : PROGRESS: saving document #0
2017-03-02 10:31:23,393 : INFO : saved 9x12 matrix, density=25.926% (28/108)
2017-03-02 10:31:23,395 : INFO : saving MmCorpus index to corpus.mm.index


In [15]:
class MyCorpus(object):
    def __iter__(self):
        for line in open('fomc_sample.txt'):
            # assume there's one document per line, tokens separated by whitespace
            yield dictionary.doc2bow(line.lower().split())

In [16]:
corpus_memory_friendly = MyCorpus()

In [17]:
for vector in corpus_memory_friendly:  # load one vector into memory at a time
    print(vector)

[]
[]
[(6, 1)]
[]
[]
[(5, 1)]
[]
[(6, 2)]
[(5, 1)]
[]
[]
[]
[]
[(5, 1)]
[]
[]
[]
[(5, 1)]
[]
[]
[]
[]
[(2, 1)]
[(7, 1)]
[(5, 1)]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[(6, 1)]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[(5, 1)]
[]
[]
[]
[]


In [40]:
from six import iteritems
import re

# collect statistics about all tokens
words = [[word for word in line.lower().split() if re.match('^[a-zA-Z]+$', word)] for line in open('fomc_sample.txt')]
dictionary = corpora.Dictionary(words)

# remove stop words and words that appear only once
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist 
            if stopword in dictionary.token2id]
once_ids = [tokenid for tokenid, docfreq in iteritems(dictionary.dfs) if docfreq == 1]

# remove stop words and words that appear only once
dictionary.filter_tokens(stop_ids + once_ids)

# remove gaps in id sequence after words that were removed
dictionary.compactify()
print(dictionary)

2017-03-02 11:19:17,914 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2017-03-02 11:19:17,921 : INFO : built Dictionary(1018 unique tokens: ['term', 'experimental', 'ever', 'filling', 'application']...) from 54 documents (total 3872 corpus positions)


Dictionary(409 unique tokens: ['term', 'office', 'continue', 'such', 'strong']...)


In [47]:
corpus = [dictionary.doc2bow(line.lower().split()) for line in open('fomc_sample.txt')]

In [48]:
print(list(corpus))

[[(256, 1), (286, 1), (376, 1)], [(48, 1), (126, 1), (129, 1), (132, 1), (134, 1), (136, 2), (145, 1), (164, 1), (211, 1), (219, 1), (254, 1), (267, 1), (280, 1), (286, 1), (309, 1), (310, 1), (319, 1), (329, 2), (331, 1), (338, 1), (349, 1), (361, 1), (384, 1), (404, 1)], [(5, 1), (6, 9), (7, 1), (16, 1), (25, 1), (33, 1), (44, 1), (45, 1), (57, 1), (60, 2), (63, 2), (70, 1), (71, 2), (75, 1), (78, 1), (83, 2), (85, 1), (86, 2), (89, 2), (90, 1), (91, 1), (97, 2), (107, 1), (112, 1), (125, 1), (132, 1), (134, 2), (140, 1), (143, 2), (152, 5), (155, 3), (156, 5), (157, 1), (158, 1), (160, 1), (163, 1), (166, 1), (174, 1), (175, 1), (176, 1), (181, 1), (186, 1), (195, 1), (211, 1), (215, 1), (219, 2), (222, 1), (223, 2), (224, 1), (240, 1), (243, 1), (249, 1), (253, 1), (258, 1), (260, 1), (263, 2), (264, 1), (267, 1), (286, 1), (291, 1), (292, 1), (296, 1), (300, 1), (313, 1), (316, 1), (338, 1), (341, 1), (349, 1), (351, 1), (356, 5), (362, 2), (376, 1), (392, 1), (398, 3), (402, 1), 

In [49]:
tfidf = models.TfidfModel(corpus)

2017-03-02 11:23:04,130 : INFO : collecting document frequencies
2017-03-02 11:23:04,132 : INFO : PROGRESS: processing document #0
2017-03-02 11:23:04,134 : INFO : calculating IDF weights for 54 documents and 408 features (1744 matrix non-zeros)


In [51]:
tfidf

In [52]:
doc_bow = [(0, 1), (1, 1)]
print(tfidf[doc_bow]) # step 2 -- use the model to transform vectors

[(0, 0.7518397621838828), (1, 0.6593458667492219)]


In [54]:
corpus_tfidf = tfidf[corpus]
for doc in corpus_tfidf:
    print(doc)

[(256, 0.8297530086695203), (286, 0.441811077381648), (376, 0.3410467951861538)]
[(48, 0.2613377516713744), (126, 0.2613377516713744), (129, 0.17422516778091626), (132, 0.06767601749748516), (134, 0.12616266512900862), (136, 0.30282781899483013), (145, 0.2613377516713744), (164, 0.17422516778091626), (211, 0.08282541598125796), (219, 0.10704013198210043), (254, 0.2613377516713744), (267, 0.18868204403751382), (280, 0.20637583058439474), (286, 0.07120074404202895), (309, 0.2291870888678959), (310, 0.2291870888678959), (319, 0.2291870888678959), (329, 0.3484503355618325), (331, 0.12616266512900862), (338, 0.08282541598125796), (349, 0.09645198841043541), (361, 0.2613377516713744), (384, 0.17422516778091626), (404, 0.20637583058439474)]
[(5, 0.09695990218141584), (6, 0.3982347515195951), (7, 0.09695990218141584), (16, 0.09695990218141584), (25, 0.09695990218141584), (33, 0.07000363482223541), (44, 0.07000363482223541), (45, 0.09695990218141584), (57, 0.08503156385082165), (60, 0.193919804

In [57]:
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=4) # initialize an LSI transformation
corpus_lsi = lsi[corpus_tfidf] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

2017-03-02 11:36:09,177 : INFO : using serial LSI version on this node
2017-03-02 11:36:09,179 : INFO : updating model with new documents
2017-03-02 11:36:09,185 : INFO : preparing a new chunk of documents
2017-03-02 11:36:09,187 : INFO : using 100 extra samples and 2 power iterations
2017-03-02 11:36:09,189 : INFO : 1st phase: constructing (409, 104) action matrix
2017-03-02 11:36:09,192 : INFO : orthonormalizing (409, 104) action matrix
2017-03-02 11:36:09,201 : INFO : 2nd phase: running dense svd on (104, 54) matrix
2017-03-02 11:36:09,208 : INFO : computing the final decomposition
2017-03-02 11:36:09,210 : INFO : keeping 4 factors (discarding 77.632% of energy spectrum)
2017-03-02 11:36:09,212 : INFO : processed documents up to #54
2017-03-02 11:36:09,215 : INFO : topic #0(2.176): -0.191*"that" + -0.170*"is" + -0.166*"rule" + -0.151*"policy" + -0.149*"has" + -0.133*"rules" + -0.130*"rate" + -0.123*"are" + -0.122*"john" + -0.122*"monetary"
2017-03-02 11:36:09,217 : INFO : topic #1(1

In [58]:
for doc in corpus_lsi: # both bow->tfidf and tfidf->lsi transformations are actually executed here, on the fly
    print(doc)

[(0, -0.18935629564720852), (1, 0.10024747186795817), (2, -0.22373881682708946), (3, 0.1355082276274242)]
[(0, -0.22294451822801806), (1, -0.04280409200876014), (2, 0.034997448008654078), (3, -0.046117192664416856)]
[(0, -0.37431014388403006), (1, -0.1568616245995392), (2, 0.1297474156501221), (3, -0.013490899441801048)]
[(0, -0.3792097605680908), (1, -0.17092198300118192), (2, 0.12681641677698485), (3, 0.0093467783153789816)]
[(0, -0.23231186586635461), (1, -0.12224454460420935), (2, 0.073420715497605205), (3, 0.016822772303389015)]
[(0, -0.44087219165514813), (1, -0.20123164627047416), (2, 0.13792948359623161), (3, -0.0007700892098716737)]
[(0, -0.30688604490264421), (1, -0.11919620833061421), (2, -0.00060728560622268506), (3, 0.033258287546919445)]
[(0, -0.37863941860711092), (1, -0.16626478200999492), (2, 0.089678421360810667), (3, 0.089047202042939977)]
[(0, -0.37511840079889325), (1, -0.1659139234575068), (2, 0.069207321727264501), (3, 0.080939554571832883)]
[(0, -0.3333667376232

In [59]:
model2 = models.LdaModel(corpus, id2word=dictionary, num_topics=4)
corpus_lsi = model2[corpus_tfidf]
for doc in corpus_lsi:
    print(doc)

2017-03-02 11:51:20,404 : INFO : using symmetric alpha at 0.25
2017-03-02 11:51:20,407 : INFO : using symmetric eta at 0.0024449877750611247
2017-03-02 11:51:20,409 : INFO : using serial LDA version on this node
2017-03-02 11:51:20,422 : INFO : running online LDA training, 4 topics, 1 passes over the supplied corpus of 54 documents, updating model once every 54 documents, evaluating perplexity every 54 documents, iterating 50x with a convergence threshold of 0.001000
2017-03-02 11:51:20,423 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2017-03-02 11:51:20,594 : INFO : -6.952 per-word bound, 123.8 perplexity estimate based on a held-out corpus of 54 documents with 2342 words
2017-03-02 11:51:20,595 : INFO : PROGRESS: pass 0, at document #54/54
2017-03-02 11:51:20,672 : INFO : topic #0 (0.250): 0.030*"is" + 0.024*"that" + 0.018*"has" + 0.017*"are" + 0.015*"as" + 0.015*"inflation" + 0.013*"have" + 0.013

[(0, 0.097367372248302358), (1, 0.10072137582767571), (2, 0.10465400081146733), (3, 0.69725725111255443)]
[(0, 0.046977060793909495), (1, 0.047759538136172397), (2, 0.04740793488358204), (3, 0.85785546618633601)]
[(0, 0.9054384468757497), (1, 0.031190330844804017), (2, 0.031601501101409774), (3, 0.031769721178036489)]
[(0, 0.040062128353395933), (1, 0.038939962368809283), (2, 0.038655126068542142), (3, 0.88234278320925263)]
[(0, 0.88071315269835693), (1, 0.039717304633170254), (2, 0.039632847577388407), (3, 0.039936695091084308)]
[(0, 0.92008039098266015), (1, 0.02655208782843094), (2, 0.02663619725052856), (3, 0.02673132393838026)]
[(0, 0.039093323585079824), (1, 0.037849209883897915), (2, 0.037413254597830431), (3, 0.88564421193319187)]
[(0, 0.88506118773816633), (1, 0.038509252052600085), (2, 0.037666099219794122), (3, 0.038763460989439544)]
[(0, 0.89838297680963475), (1, 0.034177218734302829), (2, 0.034017434412286773), (3, 0.033422370043775629)]
[(0, 0.039661629289124527), (1, 0.0